In [ ]:
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from pyspark.sql import SparkSession

sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session

# Tabela 1
table_name1 = "dim_nome_filme"
datasource1 = glueContext.create_dynamic_frame.from_catalog(database="trusted-csv-json-modelagem-refined", table_name=table_name1)
dim_nome_filme_df = datasource1.toDF()

# Tabela 2
table_name2 = "dim_tempo"
datasource1 = glueContext.create_dynamic_frame.from_catalog(database="trusted-csv-json-modelagem-refined", table_name=table_name1)
dim_tempo_df = datasource1.toDF()

# Tabela 3
table_name3 = "fact_popularidade"
datasource1 = glueContext.create_dynamic_frame.from_catalog(database="trusted-csv-json-modelagem-refined", table_name=table_name1)
fact_popularidade_df = datasource1.toDF()

# Transformação
# Realizando a junção das tabelas
joined_df = dim_nome_filme_df.join(fact_popularidade_df, "id_csv") \
    .join(dim_tempo_df, "id_csv") \
    .select(
        dim_nome_filme_df.tituloPincipal,
        dim_nome_filme_df.genero,
        fact_popularidade_df.notamedia,
        fact_popularidade_df.numerovotos,
        fact_popularidade_df.popularity,
        dim_tempo_df.anolancamento,
        dim_tempo_df.release_date,
        dim_nome_filme_df.overview  
    )

# Fazendo a agregação e ordenação
aggregated_order_df = joined_df.groupBy(
    "anolancamento",
    "tituloPincipal",
    "genero",
    "numerovotos",
    "popularity"
).agg(
    max("numerovotos").alias("maior_numerovotos"),
    max("popularity").alias("maior_popularity")
).orderBy(
    "maior_numerovotos",
    "maior_popularity",
    ascending=False
).limit(10)

# Caminho de saída para o join das dimensoes e fato
output_path1 = "s3://data-lake-heliton/refined/juncao-dimensoes-fato"
joined_df.write.parquet(output_path1, mode="overwrite")

# Caminho de saída para agregação e ordenação
output_path2 = "s3://data-lake-heliton/refined/agregacao-ordenacao"
aggregated_order_df.write.parquet(output_path2, mode="overwrite")

In [ ]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext, SparkConf
from awsglue.context import GlueContext
from awsglue.job import Job
from pyspark.sql.functions import from_json, col, explode, array_contains, concat_ws, to_date
from datetime import datetime
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType

## @params: [JOB_NAME]
args = getResolvedOptions(sys.argv, ['JOB_NAME', 'S3_RAW_PATH', 'S3_TRUSTED_PATH'])

conf = SparkConf().setAppName("TMDB").setMaster("local")
sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)
job.init(args['JOB_NAME'], args)


# # Salva o resultado em formato Parquet no caminho especificado
df.write.mode("overwrite").parquet(args['S3_TRUSTED_PATH'])

job.commit()

In [ ]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext, SparkConf
from awsglue.context import GlueContext
from awsglue.job import Job
from pyspark.sql.functions import from_json, col, explode, array_contains, concat_ws, to_date
from datetime import datetime
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType

## @params: [JOB_NAME, S3_RAW_PATH, S3_TRUSTED_PATH]
args = getResolvedOptions(sys.argv, ['JOB_NAME', 'S3_RAW_PATH', 'S3_TRUSTED_PATH'])

conf = SparkConf().setAppName("TMDB").setMaster("local")
sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)
job.init(args['JOB_NAME'], args)

# Realize o processamento e armazene em um DataFrame processado

# Caminho de entrada para os dados brutos
input_path = args['S3_RAW_PATH']

# Faça a leitura dos dados brutos
df_raw = spark.read.parquet(input_path)

# Faça as transformações necessárias no DataFrame df_raw para obter o df_processado

# Caminho de saída para o DataFrame processado
output_path = args['S3_TRUSTED_PATH']

# Salve o resultado em formato Parquet no caminho especificado
df_processado.write.mode("overwrite").parquet(output_path)

job.commit()

In [ ]:
from pyspark.sql import SparkSession

# Crie uma instância do SparkSession
spark = SparkSession.builder.getOrCreate()

# Leia a tabela do catálogo do Glue
database_name = "trusted-csv-json-modelagem-refined"
table_name = "dim_nome_filme"
df_dim_nome_filme = spark.table(f"{database_name}.{table_name}")

# Faça o mesmo para as outras tabelas
table_name2 = "dim_tempo"
df_dim_tempo = spark.table(f"{database_name}.{table_name2}")

table_name3 = "fact_popularidade"
df_fact_popularidade = spark.table(f"{database_name}.{table_name3}")

# Faça as transformações necessárias
# ...

# Salve os resultados no novo bucket S3
output_bucket = "s3://seu-novo-bucket/"
output_path = output_bucket + "caminho-de-saida/"
df_resultado.write.parquet(output_path, mode="overwrite")

In [ ]:
# Configurar o caminho do armazém do Spark
spark.conf.set("spark.sql.warehouse.dir", "s3://seu-bucket/spark-warehouse/")

# Leia a tabela do catálogo do Glue
database_name = "trusted-csv-json-modelagem-refined"
table_name = "dim_nome_filme"
df_dim_nome_filme = spark.table(f"{database_name}.{table_name}")

# Faça o mesmo para as outras tabelas
table_name2 = "dim_tempo"
df_dim_tempo = spark.table(f"{database_name}.{table_name2}")

table_name3 = "fact_popularidade"
df_fact_popularidade = spark.table(f"{database_name}.{table_name3}")

In [ ]:
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from pyspark.sql import SparkSession

sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session

# Carregar tabelas como DynamicFrames
table_name1 = "dim_nome_filme"
table_name2 = "dim_tempo"
table_name3 = "fact_popularidade"

datasource1 = glueContext.create_dynamic_frame.from_catalog(database="trusted-csv-json-modelagem-refined", table_name=table_name1)
datasource2 = glueContext.create_dynamic_frame.from_catalog(database="trusted-csv-json-modelagem-refined", table_name=table_name2)
datasource3 = glueContext.create_dynamic_frame.from_catalog(database="trusted-csv-json-modelagem-refined", table_name=table_name3)

# Converter DynamicFrames em DataFrames
df_dim_nome_filme = datasource1.toDF()
df_dim_tempo = datasource2.toDF()
df_fact_popularidade = datasource3.toDF()

# Realizar a junção das tabelas e selecionar as colunas desejadas
joined_df = df_dim_nome_filme.join(df_fact_popularidade, "id_csv") \
    .join(df_dim_tempo, "id_csv") \
    .select(
        df_dim_nome_filme.tituloPincipal,
        df_dim_nome_filme.genero,
        df_fact_popularidade.notamedia,
        df_fact_popularidade.numerovotos,
        df_fact_popularidade.popularity,
        df_dim_tempo.anolancamento,
        df_dim_tempo.release_date,
        df_dim_nome_filme.overview  
    )

# Realizar a agregação e ordenação
aggregated_order_df = joined_df.groupBy(
    "anolancamento",
    "tituloPincipal",
    "genero",
    "numerovotos",
    "popularity"
).agg(
    max("numerovotos").alias("maior_numerovotos"),
    max("popularity").alias("maior_popularity")
).orderBy(
    "maior_numerovotos",
    "maior_popularity",
    ascending=False
).limit(10)

# Caminho de saída para o join das dimensões e fato
output_path1 = "s3://data-lake-heliton/refined/juncao-dimensoes-fato"
joined_df.write.parquet(output_path1, mode="overwrite")

# Caminho de saída para a agregação e ordenação
output_path2 = "s3://data-lake-heliton/refined/agregacao-ordenacao"
aggregated_order_df.write.parquet(output_path2, mode="overwrite")

In [ ]:
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from pyspark.sql import SparkSession

sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session

# Carregar tabelas como DynamicFrames
table_name1 = "dim_nome_filme"
table_name2 = "dim_tempo"
table_name3 = "fact_popularidade"

datasource1 = glueContext.create_dynamic_frame.from_catalog(database="trusted-csv-json-modelagem-refined", table_name=table_name1)
datasource2 = glueContext.create_dynamic_frame.from_catalog(database="trusted-csv-json-modelagem-refined", table_name=table_name2)
datasource3 = glueContext.create_dynamic_frame.from_catalog(database="trusted-csv-json-modelagem-refined", table_name=table_name3)

# Realizar a junção das tabelas e selecionar as colunas desejadas
joined_df = datasource1.join(datasource3, "id_csv") \
    .join(datasource2, "id_csv") \
    .select(
        "dim_nome_filme.tituloPincipal",
        "dim_nome_filme.genero",
        "fact_popularidade.notamedia",
        "fact_popularidade.numerovotos",
        "fact_popularidade.popularity",
        "dim_tempo.anolancamento",
        "dim_tempo.release_date",
        "dim_nome_filme.overview"
    )

# Realizar a agregação e ordenação
aggregated_order_df = joined_df.groupBy(
    "anolancamento",
    "tituloPincipal",
    "genero",
    "numerovotos",
    "popularity"
).agg(
    max("numerovotos").alias("maior_numerovotos"),
    max("popularity").alias("maior_popularity")
).orderBy(
    "maior_numerovotos",
    "maior_popularity",
    ascending=False
).limit(10)

# Caminho de saída para o join das dimensões e fato
output_path1 = "s3://data-lake-heliton/refined/juncao-dimensoes-fato"
glueContext.write_dynamic_frame.from_options(joined_df, connection_type="s3", connection_options={"path": output_path1}, format="parquet", transformation_ctx="output1")

# Caminho de saída para a agregação e ordenação
output_path2 = "s3://data-lake-heliton/refined/agregacao-ordenacao"
glueContext.write_dynamic_frame.from_options(aggregated_order_df, connection_type="s3", connection_options={"path": output_path2}, format="parquet", transformation_ctx="output2")

In [ ]:
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from pyspark.sql import SparkSession

sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session

# Especifique os detalhes da tabela
database_name = "trusted-csv-json-modelagem-refined"
table_name = "dim_nome_filme"
table_path = "s3://data-lake-heliton/trusted/csv_imdb/movies/"  # Especifique o caminho correto para os dados

# Crie a tabela no Glue Catalog
glueContext.create_dynamic_frame.from_catalog(database=database_name, table_name=table_name, transformation_ctx="datasource0")
datasource0.toDF().write.format("parquet").mode("overwrite").save(table_path)

In [ ]:
from pyspark.sql import SparkSession

# Crie a sessão Spark
spark = SparkSession.builder \
    .appName("Create Tables") \
    .getOrCreate()

# Especifique os detalhes da tabela
database_name = "trusted-csv-json-modelagem-refined"
table_name = "dim_nome_filme"

# Crie o DataFrame correspondente à visualização
df_dim_nome_filme = spark.sql("SELECT tb_csv.id as id_csv, "
                              "tb_json.id as id_json, "
                              "tb_csv.tituloPincipal, "
                              "tb_csv.genero, "
                              "tb_json.overview "
                              "FROM tb_csv "
                              "JOIN tb_json "
                              "ON tb_csv.titulopincipal = tb_json.title")

# Salve o DataFrame como uma tabela no Glue Catalog
df_dim_nome_filme.write \
    .format("parquet") \
    .mode("overwrite") \
    .option("path", f"s3://data-lake-heliton/trusted/csv_imdb/movies/{table_name}") \
    .saveAsTable(f"{database_name}.{table_name}")

In [ ]:
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from pyspark.sql import SparkSession

sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session

# Carregar as tabelas existentes
table_name1 = "tb_csv"
table_name2 = "tb_json"

df_tb_csv = spark.table(table_name1)
df_tb_json = spark.table(table_name2)

# Criar as dimensões e a tabela de fato
df_dim_nome_filme = df_tb_csv.join(df_tb_json, df_tb_csv["titulopincipal"] == df_tb_json["title"]) \
    .select(df_tb_csv["id"].alias("id_csv"),
            df_tb_json["id"].alias("id_json"),
            df_tb_csv["tituloPincipal"],
            df_tb_csv["genero"],
            df_tb_json["overview"])

df_dim_tempo = df_tb_csv.join(df_tb_json, df_tb_csv["titulopincipal"] == df_tb_json["title"]) \
    .select(df_tb_csv["id"].alias("id_csv"),
            df_tb_json["id"].alias("id_json"),
            df_tb_csv["anoLancamento"],
            df_tb_json["release_date"])

df_fact_popularidade = df_tb_csv.join(df_tb_json, df_tb_csv["titulopincipal"] == df_tb_json["title"]) \
    .select(df_tb_csv["id"].alias("id_csv"),
            df_tb_json["id"].alias("id_json"),
            df_tb_csv["notaMedia"],
            df_tb_csv["numeroVotos"],
            df_tb_json["popularity"])

# Realizar operações com as dimensões e a tabela de fato
# ...

# Salvar as dimensões e a tabela de fato em um local de sua escolha
output_path_dim_nome_filme = "s3://data-lake-heliton/refined/dim_nome_filme/"
output_path_dim_tempo = "s3://data-lake-heliton/refined/dim_tempo/"
output_path_fact_popularidade = "s3://data-lake-heliton/refined//fact_popularidade/"

df_dim_nome_filme.write.parquet(output_path_dim_nome_filme, mode="overwrite")
df_dim_tempo.write.parquet(output_path_dim_tempo, mode="overwrite")
df_fact_popularidade.write.parquet(output_path_fact_popularidade, mode="overwrite")